In [31]:
edges = read.csv('../data/edgelist_2011.csv')
nodes = read.csv('../data/nodelist_2011.csv')

In [32]:
n_countries = dim(nodes)[1]
dyads = matrix(0, nrow = n_countries, ncol = n_countries)

In [33]:
for (i in 1:n_countries) {
    for (j in 1:n_countries) {
        if (sum(edges$source == nodes$country_iso3[i] & edges$target == nodes$country_iso3[j])) {
            dyads[i, j] = 1
        }
    }
}

In [42]:
dyads.flatten = as.vector(t(dyads))